# Abstraction in Python

---

## Table of Contents
1. What is Abstraction?
2. Abstract Base Classes (ABC)
3. Abstract Methods
4. Abstract Properties
5. Concrete Methods in Abstract Classes
6. Multiple Inheritance with ABCs
7. Built-in ABCs in Python
8. Practical Examples
9. Key Points
10. Practice Exercises

---

## 1. What is Abstraction?

**Abstraction** hides complex implementation details and shows only essential features.

**Key Concepts:**
- **Abstract Class**: Cannot be instantiated, serves as a blueprint
- **Abstract Method**: Declared but not implemented in abstract class
- **Concrete Class**: Implements all abstract methods, can be instantiated

**Benefits:**
- Enforces a consistent interface across subclasses
- Hides complexity from users
- Makes code more maintainable and extensible

In [ ]:
# Without abstraction - no guaranteed interface
class Dog:
    def bark(self):
        return "Woof!"

class Cat:
    def meow(self):  # Different method name!
        return "Meow!"

class Cow:
    def speak(self):  # Yet another name!
        return "Moo!"

# Hard to use polymorphically
animals = [Dog(), Cat(), Cow()]
# for animal in animals:
#     print(animal.speak())  # Fails! No consistent interface

In [ ]:
# With abstraction - guaranteed interface
from abc import ABC, abstractmethod

class Animal(ABC):
    @abstractmethod
    def speak(self):
        pass

class Dog(Animal):
    def speak(self):
        return "Woof!"

class Cat(Animal):
    def speak(self):
        return "Meow!"

class Cow(Animal):
    def speak(self):
        return "Moo!"

# Now polymorphism works!
animals = [Dog(), Cat(), Cow()]
for animal in animals:
    print(f"{animal.__class__.__name__}: {animal.speak()}")

---

## 2. Abstract Base Classes (ABC)

The `abc` module provides tools for creating abstract classes.

In [ ]:
from abc import ABC, abstractmethod

# Abstract class - cannot be instantiated
class Shape(ABC):
    @abstractmethod
    def area(self):
        """Calculate and return the area."""
        pass
    
    @abstractmethod
    def perimeter(self):
        """Calculate and return the perimeter."""
        pass

# Try to instantiate abstract class
try:
    shape = Shape()
except TypeError as e:
    print(f"Cannot instantiate: {e}")

In [ ]:
# Concrete class must implement all abstract methods
import math

class Circle(Shape):
    def __init__(self, radius):
        self.radius = radius
    
    def area(self):
        return math.pi * self.radius ** 2
    
    def perimeter(self):
        return 2 * math.pi * self.radius

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height
    
    def area(self):
        return self.width * self.height
    
    def perimeter(self):
        return 2 * (self.width + self.height)

# Can instantiate concrete classes
circle = Circle(5)
rect = Rectangle(4, 6)

print(f"Circle: area={circle.area():.2f}, perimeter={circle.perimeter():.2f}")
print(f"Rectangle: area={rect.area()}, perimeter={rect.perimeter()}")

In [ ]:
# Incomplete implementation - still abstract
class IncompleteShape(Shape):
    def area(self):
        return 0
    # Missing perimeter() implementation!

try:
    incomplete = IncompleteShape()
except TypeError as e:
    print(f"Cannot instantiate: {e}")

---

## 3. Abstract Methods

In [ ]:
from abc import ABC, abstractmethod

class Database(ABC):
    """Abstract database interface."""
    
    @abstractmethod
    def connect(self):
        """Establish database connection."""
        pass
    
    @abstractmethod
    def disconnect(self):
        """Close database connection."""
        pass
    
    @abstractmethod
    def execute(self, query):
        """Execute a query."""
        pass

class MySQLDatabase(Database):
    def connect(self):
        return "Connected to MySQL"
    
    def disconnect(self):
        return "Disconnected from MySQL"
    
    def execute(self, query):
        return f"MySQL executing: {query}"

class PostgreSQLDatabase(Database):
    def connect(self):
        return "Connected to PostgreSQL"
    
    def disconnect(self):
        return "Disconnected from PostgreSQL"
    
    def execute(self, query):
        return f"PostgreSQL executing: {query}"

# Use any database with same interface
def run_query(db: Database, query: str):
    print(db.connect())
    print(db.execute(query))
    print(db.disconnect())

print("=== MySQL ===")
run_query(MySQLDatabase(), "SELECT * FROM users")
print("\n=== PostgreSQL ===")
run_query(PostgreSQLDatabase(), "SELECT * FROM users")

In [ ]:
# Abstract method with default implementation (call super)
from abc import ABC, abstractmethod

class Serializer(ABC):
    @abstractmethod
    def serialize(self, data):
        """Subclasses can call super().serialize() for preprocessing."""
        # Preprocessing: ensure data is dict
        if not isinstance(data, dict):
            raise ValueError("Data must be a dictionary")
        return data

class JSONSerializer(Serializer):
    def serialize(self, data):
        import json
        validated = super().serialize(data)  # Call abstract method!
        return json.dumps(validated)

class XMLSerializer(Serializer):
    def serialize(self, data):
        validated = super().serialize(data)
        elements = [f"<{k}>{v}</{k}>" for k, v in validated.items()]
        return f"<data>{''.join(elements)}</data>"

json_s = JSONSerializer()
xml_s = XMLSerializer()

data = {"name": "Alice", "age": 30}
print(f"JSON: {json_s.serialize(data)}")
print(f"XML: {xml_s.serialize(data)}")

---

## 4. Abstract Properties

In [ ]:
from abc import ABC, abstractmethod

class Vehicle(ABC):
    @property
    @abstractmethod
    def wheels(self):
        """Number of wheels."""
        pass
    
    @property
    @abstractmethod
    def fuel_type(self):
        """Type of fuel used."""
        pass
    
    @abstractmethod
    def drive(self):
        pass

class Car(Vehicle):
    @property
    def wheels(self):
        return 4
    
    @property
    def fuel_type(self):
        return "Gasoline"
    
    def drive(self):
        return f"Driving car with {self.wheels} wheels on {self.fuel_type}"

class Motorcycle(Vehicle):
    @property
    def wheels(self):
        return 2
    
    @property
    def fuel_type(self):
        return "Gasoline"
    
    def drive(self):
        return f"Riding motorcycle with {self.wheels} wheels"

class ElectricScooter(Vehicle):
    @property
    def wheels(self):
        return 2
    
    @property
    def fuel_type(self):
        return "Electric"
    
    def drive(self):
        return f"Scooting on {self.fuel_type} power"

vehicles = [Car(), Motorcycle(), ElectricScooter()]
for v in vehicles:
    print(f"{v.__class__.__name__}: {v.wheels} wheels, {v.fuel_type}")
    print(f"  {v.drive()}")

In [ ]:
# Abstract property with setter
from abc import ABC, abstractmethod

class Account(ABC):
    @property
    @abstractmethod
    def balance(self):
        pass
    
    @balance.setter
    @abstractmethod
    def balance(self, value):
        pass

class SavingsAccount(Account):
    def __init__(self, initial_balance):
        self._balance = initial_balance
    
    @property
    def balance(self):
        return self._balance
    
    @balance.setter
    def balance(self, value):
        if value < 0:
            raise ValueError("Balance cannot be negative")
        self._balance = value

acc = SavingsAccount(1000)
print(f"Balance: ${acc.balance}")
acc.balance = 1500
print(f"New balance: ${acc.balance}")

---

## 5. Concrete Methods in Abstract Classes

Abstract classes can have concrete (implemented) methods.

In [ ]:
from abc import ABC, abstractmethod

class Report(ABC):
    def __init__(self, title):
        self.title = title
    
    @abstractmethod
    def generate_content(self):
        """Abstract: subclasses define content generation."""
        pass
    
    # Concrete methods - shared implementation
    def add_header(self):
        return f"=== {self.title} ==="
    
    def add_footer(self):
        from datetime import datetime
        return f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M')}\n{'=' * 30}"
    
    def generate(self):
        """Template method pattern."""
        return f"{self.add_header()}\n{self.generate_content()}\n{self.add_footer()}"

class SalesReport(Report):
    def __init__(self, title, sales_data):
        super().__init__(title)
        self.sales_data = sales_data
    
    def generate_content(self):
        lines = []
        total = 0
        for product, amount in self.sales_data.items():
            lines.append(f"  {product}: ${amount}")
            total += amount
        lines.append(f"  TOTAL: ${total}")
        return "\n".join(lines)

class InventoryReport(Report):
    def __init__(self, title, inventory):
        super().__init__(title)
        self.inventory = inventory
    
    def generate_content(self):
        lines = []
        for item, qty in self.inventory.items():
            status = "LOW" if qty < 10 else "OK"
            lines.append(f"  {item}: {qty} units [{status}]")
        return "\n".join(lines)

# Generate reports
sales = SalesReport("Monthly Sales", {"Laptops": 5000, "Phones": 3000, "Tablets": 2000})
print(sales.generate())

print()

inventory = InventoryReport("Stock Check", {"Laptops": 15, "Phones": 8, "Tablets": 25})
print(inventory.generate())

In [ ]:
# Abstract class with __init__ and common functionality
from abc import ABC, abstractmethod

class Employee(ABC):
    def __init__(self, name, employee_id):
        self.name = name
        self.employee_id = employee_id
    
    @abstractmethod
    def calculate_pay(self):
        pass
    
    # Concrete method
    def get_info(self):
        return f"{self.employee_id}: {self.name}"

class SalariedEmployee(Employee):
    def __init__(self, name, employee_id, annual_salary):
        super().__init__(name, employee_id)
        self.annual_salary = annual_salary
    
    def calculate_pay(self):
        return self.annual_salary / 12

class HourlyEmployee(Employee):
    def __init__(self, name, employee_id, hourly_rate, hours_worked):
        super().__init__(name, employee_id)
        self.hourly_rate = hourly_rate
        self.hours_worked = hours_worked
    
    def calculate_pay(self):
        overtime = max(0, self.hours_worked - 40)
        regular = min(self.hours_worked, 40)
        return (regular * self.hourly_rate) + (overtime * self.hourly_rate * 1.5)

employees = [
    SalariedEmployee("Alice", "E001", 60000),
    HourlyEmployee("Bob", "E002", 25, 45)
]

for emp in employees:
    print(f"{emp.get_info()} - Monthly Pay: ${emp.calculate_pay():.2f}")

---

## 6. Multiple Inheritance with ABCs

In [ ]:
from abc import ABC, abstractmethod

# Multiple abstract base classes
class Printable(ABC):
    @abstractmethod
    def print_details(self):
        pass

class Saveable(ABC):
    @abstractmethod
    def save(self):
        pass
    
    @abstractmethod
    def load(self):
        pass

class Exportable(ABC):
    @abstractmethod
    def export(self, format):
        pass

# Concrete class implementing multiple ABCs
class Document(Printable, Saveable, Exportable):
    def __init__(self, title, content):
        self.title = title
        self.content = content
    
    def print_details(self):
        return f"Document: {self.title}\nContent: {self.content[:50]}..."
    
    def save(self):
        return f"Saving document '{self.title}' to database"
    
    def load(self):
        return f"Loading document '{self.title}' from database"
    
    def export(self, format):
        return f"Exporting '{self.title}' as {format}"

doc = Document("Report", "This is a long report content that goes on and on...")
print(doc.print_details())
print(doc.save())
print(doc.export("PDF"))

In [ ]:
# Interface segregation with multiple ABCs
from abc import ABC, abstractmethod

class Flyable(ABC):
    @abstractmethod
    def fly(self):
        pass

class Swimmable(ABC):
    @abstractmethod
    def swim(self):
        pass

class Walkable(ABC):
    @abstractmethod
    def walk(self):
        pass

# Animals implement only relevant interfaces
class Duck(Flyable, Swimmable, Walkable):
    def fly(self):
        return "Duck flying"
    def swim(self):
        return "Duck swimming"
    def walk(self):
        return "Duck walking"

class Penguin(Swimmable, Walkable):  # Can't fly!
    def swim(self):
        return "Penguin swimming fast!"
    def walk(self):
        return "Penguin waddling"

class Eagle(Flyable):  # Only flies
    def fly(self):
        return "Eagle soaring high"

# Type checking
animals = [Duck(), Penguin(), Eagle()]

print("=== Swimming Animals ===")
for animal in animals:
    if isinstance(animal, Swimmable):
        print(f"  {animal.__class__.__name__}: {animal.swim()}")

print("\n=== Flying Animals ===")
for animal in animals:
    if isinstance(animal, Flyable):
        print(f"  {animal.__class__.__name__}: {animal.fly()}")

---

## 7. Built-in ABCs in Python

Python provides ABCs in the `collections.abc` module.

In [ ]:
from collections.abc import Iterable, Iterator, Sequence, Mapping

# Check if objects implement interfaces
print(f"list is Iterable: {isinstance([], Iterable)}")
print(f"list is Sequence: {isinstance([], Sequence)}")
print(f"dict is Mapping: {isinstance({}, Mapping)}")
print(f"str is Iterable: {isinstance('', Iterable)}")
print(f"int is Iterable: {isinstance(5, Iterable)}")

In [ ]:
# Creating custom iterable
from collections.abc import Iterable, Iterator

class CountDown(Iterable):
    def __init__(self, start):
        self.start = start
    
    def __iter__(self):
        return CountDownIterator(self.start)

class CountDownIterator(Iterator):
    def __init__(self, start):
        self.current = start
    
    def __next__(self):
        if self.current < 0:
            raise StopIteration
        value = self.current
        self.current -= 1
        return value

countdown = CountDown(5)
print(f"Is Iterable: {isinstance(countdown, Iterable)}")
print(f"Countdown: {list(countdown)}")

In [ ]:
# Custom Sequence implementation
from collections.abc import Sequence

class ImmutableList(Sequence):
    def __init__(self, items):
        self._items = list(items)
    
    def __getitem__(self, index):
        return self._items[index]
    
    def __len__(self):
        return len(self._items)

il = ImmutableList([1, 2, 3, 4, 5])

# Sequence provides these methods for free:
print(f"Length: {len(il)}")
print(f"Index 2: {il[2]}")
print(f"Contains 3: {3 in il}")
print(f"Index of 4: {il.index(4)}")
print(f"Count of 2: {il.count(2)}")
print(f"Reversed: {list(reversed(il))}")

In [ ]:
# Custom Mapping implementation
from collections.abc import Mapping

class FrozenDict(Mapping):
    def __init__(self, data):
        self._data = dict(data)
    
    def __getitem__(self, key):
        return self._data[key]
    
    def __iter__(self):
        return iter(self._data)
    
    def __len__(self):
        return len(self._data)

fd = FrozenDict({"a": 1, "b": 2, "c": 3})

# Mapping provides these for free:
print(f"Keys: {list(fd.keys())}")
print(f"Values: {list(fd.values())}")
print(f"Items: {list(fd.items())}")
print(f"Get 'a': {fd.get('a')}")
print(f"Get 'x' with default: {fd.get('x', 'not found')}")

---

## 8. Practical Examples

In [ ]:
# Plugin system using abstraction
from abc import ABC, abstractmethod

class Plugin(ABC):
    @property
    @abstractmethod
    def name(self):
        pass
    
    @abstractmethod
    def execute(self, data):
        pass

class UppercasePlugin(Plugin):
    @property
    def name(self):
        return "Uppercase"
    
    def execute(self, data):
        return data.upper()

class ReversePlugin(Plugin):
    @property
    def name(self):
        return "Reverse"
    
    def execute(self, data):
        return data[::-1]

class WordCountPlugin(Plugin):
    @property
    def name(self):
        return "Word Count"
    
    def execute(self, data):
        return f"Words: {len(data.split())}"

# Plugin runner
class PluginRunner:
    def __init__(self):
        self.plugins = []
    
    def register(self, plugin: Plugin):
        self.plugins.append(plugin)
    
    def run_all(self, data):
        results = {}
        for plugin in self.plugins:
            results[plugin.name] = plugin.execute(data)
        return results

runner = PluginRunner()
runner.register(UppercasePlugin())
runner.register(ReversePlugin())
runner.register(WordCountPlugin())

results = runner.run_all("Hello World")
for name, result in results.items():
    print(f"{name}: {result}")

In [ ]:
# Payment processing system
from abc import ABC, abstractmethod
from datetime import datetime

class PaymentProcessor(ABC):
    @abstractmethod
    def process_payment(self, amount):
        pass
    
    @abstractmethod
    def refund(self, transaction_id):
        pass
    
    # Concrete method
    def generate_transaction_id(self):
        return f"TXN{datetime.now().strftime('%Y%m%d%H%M%S')}"

class CreditCardProcessor(PaymentProcessor):
    def __init__(self, card_number):
        self.card_number = card_number
    
    def process_payment(self, amount):
        txn_id = self.generate_transaction_id()
        return {
            "status": "success",
            "transaction_id": txn_id,
            "method": "Credit Card",
            "amount": amount,
            "card": f"****{self.card_number[-4:]}"
        }
    
    def refund(self, transaction_id):
        return f"Refund initiated for {transaction_id}"

class PayPalProcessor(PaymentProcessor):
    def __init__(self, email):
        self.email = email
    
    def process_payment(self, amount):
        txn_id = self.generate_transaction_id()
        return {
            "status": "success",
            "transaction_id": txn_id,
            "method": "PayPal",
            "amount": amount,
            "account": self.email
        }
    
    def refund(self, transaction_id):
        return f"PayPal refund for {transaction_id}"

def checkout(processor: PaymentProcessor, amount: float):
    result = processor.process_payment(amount)
    print(f"Payment processed: {result}")
    return result

# Different processors, same interface
cc = CreditCardProcessor("1234567890123456")
pp = PayPalProcessor("user@email.com")

checkout(cc, 99.99)
checkout(pp, 49.99)

---

## 9. Key Points

1. **Abstraction**: Hide complexity, show only essential features
2. **ABC**: Use `from abc import ABC, abstractmethod`
3. **Abstract Class**: Cannot be instantiated, inherits from ABC
4. **Abstract Method**: Use `@abstractmethod`, must be implemented
5. **Abstract Property**: Combine `@property` and `@abstractmethod`
6. **Concrete Methods**: Abstract classes can have implemented methods
7. **Multiple ABCs**: Classes can implement multiple interfaces
8. **Built-in ABCs**: `collections.abc` provides Iterable, Sequence, Mapping, etc.
9. **Use Case**: Enforce consistent interfaces across implementations
10. **Design Principle**: Program to interfaces, not implementations

---

## 10. Practice Exercises

In [ ]:
# Exercise 1: Create a NotificationService ABC with:
# - Abstract method: send(recipient, message)
# - Concrete method: format_message(message) adds timestamp
# - Implementations: EmailService, SMSService, PushService

from abc import ABC, abstractmethod

class NotificationService(ABC):
    pass

# Test:
# for service in [EmailService(), SMSService(), PushService()]:
#     service.send("user", "Hello!")

In [ ]:
# Exercise 2: Create a FileHandler ABC with:
# - Abstract property: extension
# - Abstract methods: read(filename), write(filename, data)
# - Implementations: JSONHandler, CSVHandler, XMLHandler

class FileHandler(ABC):
    pass

# Test:
# handlers = [JSONHandler(), CSVHandler()]
# for h in handlers:
#     print(f"{h.extension}: {h.read('data')}")

In [ ]:
# Exercise 3: Create a game character system
# - Character ABC: abstract attack(), defend(), special_ability()
# - Concrete: health property, take_damage(amount), is_alive property
# - Implementations: Warrior, Mage, Archer

class Character(ABC):
    pass

# Test:
# warrior = Warrior("Conan", 100)
# mage = Mage("Gandalf", 80)
# print(warrior.attack(), warrior.special_ability())

In [ ]:
# Exercise 4: Create a data validator system
# - Validator ABC: abstract validate(value) returns bool
# - Implementations: EmailValidator, PhoneValidator, AgeValidator
# - CompositeValidator: combines multiple validators

class Validator(ABC):
    pass

# Test:
# email_v = EmailValidator()
# print(email_v.validate("test@example.com"))  # True
# composite = CompositeValidator([EmailValidator(), ...])

In [ ]:
# Exercise 5: Create a caching system
# - Cache ABC: abstract get(key), set(key, value), delete(key), clear()
# - Abstract property: size
# - Implementations: MemoryCache, FileCache

class Cache(ABC):
    pass

# Test:
# cache = MemoryCache()
# cache.set("key1", "value1")
# print(cache.get("key1"))  # value1
# print(cache.size)  # 1

---

## Solutions

In [ ]:
# Solution 1:
from abc import ABC, abstractmethod
from datetime import datetime

class NotificationService(ABC):
    @abstractmethod
    def send(self, recipient, message):
        pass
    
    def format_message(self, message):
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M")
        return f"[{timestamp}] {message}"

class EmailService(NotificationService):
    def send(self, recipient, message):
        formatted = self.format_message(message)
        return f"Email to {recipient}: {formatted}"

class SMSService(NotificationService):
    def send(self, recipient, message):
        formatted = self.format_message(message)
        return f"SMS to {recipient}: {formatted}"

class PushService(NotificationService):
    def send(self, recipient, message):
        formatted = self.format_message(message)
        return f"Push to {recipient}: {formatted}"

for service in [EmailService(), SMSService(), PushService()]:
    print(service.send("user@example.com", "Hello!"))

In [ ]:
# Solution 2:
from abc import ABC, abstractmethod
import json

class FileHandler(ABC):
    @property
    @abstractmethod
    def extension(self):
        pass
    
    @abstractmethod
    def read(self, filename):
        pass
    
    @abstractmethod
    def write(self, filename, data):
        pass

class JSONHandler(FileHandler):
    @property
    def extension(self):
        return ".json"
    
    def read(self, filename):
        return f"Reading JSON from {filename}{self.extension}"
    
    def write(self, filename, data):
        return f"Writing JSON to {filename}{self.extension}: {json.dumps(data)}"

class CSVHandler(FileHandler):
    @property
    def extension(self):
        return ".csv"
    
    def read(self, filename):
        return f"Reading CSV from {filename}{self.extension}"
    
    def write(self, filename, data):
        return f"Writing CSV to {filename}{self.extension}"

handlers = [JSONHandler(), CSVHandler()]
for h in handlers:
    print(f"{h.extension}: {h.read('data')}")
    print(f"  {h.write('data', {'key': 'value'})}")

In [ ]:
# Solution 3:
from abc import ABC, abstractmethod

class Character(ABC):
    def __init__(self, name, max_health):
        self.name = name
        self.max_health = max_health
        self._health = max_health
    
    @property
    def health(self):
        return self._health
    
    @property
    def is_alive(self):
        return self._health > 0
    
    def take_damage(self, amount):
        self._health = max(0, self._health - amount)
        return f"{self.name} takes {amount} damage. Health: {self._health}"
    
    @abstractmethod
    def attack(self):
        pass
    
    @abstractmethod
    def defend(self):
        pass
    
    @abstractmethod
    def special_ability(self):
        pass

class Warrior(Character):
    def attack(self):
        return f"{self.name} swings sword for 25 damage"
    def defend(self):
        return f"{self.name} raises shield"
    def special_ability(self):
        return f"{self.name} uses Berserk Rage!"

class Mage(Character):
    def attack(self):
        return f"{self.name} casts Fireball for 30 damage"
    def defend(self):
        return f"{self.name} creates Magic Shield"
    def special_ability(self):
        return f"{self.name} casts Meteor Storm!"

warrior = Warrior("Conan", 100)
mage = Mage("Gandalf", 80)

print(warrior.attack())
print(warrior.special_ability())
print(mage.attack())
print(warrior.take_damage(30))
print(f"{warrior.name} alive: {warrior.is_alive}")

In [ ]:
# Solution 4:
from abc import ABC, abstractmethod
import re

class Validator(ABC):
    @abstractmethod
    def validate(self, value):
        pass

class EmailValidator(Validator):
    def validate(self, value):
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        return bool(re.match(pattern, value))

class PhoneValidator(Validator):
    def validate(self, value):
        digits = re.sub(r'\D', '', value)
        return len(digits) == 10

class AgeValidator(Validator):
    def __init__(self, min_age=0, max_age=150):
        self.min_age = min_age
        self.max_age = max_age
    
    def validate(self, value):
        return isinstance(value, int) and self.min_age <= value <= self.max_age

class CompositeValidator(Validator):
    def __init__(self, validators):
        self.validators = validators
    
    def validate(self, value):
        return all(v.validate(value) for v in self.validators)

# Test individual validators
email_v = EmailValidator()
print(f"Email 'test@example.com': {email_v.validate('test@example.com')}")
print(f"Email 'invalid': {email_v.validate('invalid')}")

phone_v = PhoneValidator()
print(f"Phone '123-456-7890': {phone_v.validate('123-456-7890')}")

age_v = AgeValidator(18, 100)
print(f"Age 25: {age_v.validate(25)}")
print(f"Age 10: {age_v.validate(10)}")

In [ ]:
# Solution 5:
from abc import ABC, abstractmethod

class Cache(ABC):
    @property
    @abstractmethod
    def size(self):
        pass
    
    @abstractmethod
    def get(self, key):
        pass
    
    @abstractmethod
    def set(self, key, value):
        pass
    
    @abstractmethod
    def delete(self, key):
        pass
    
    @abstractmethod
    def clear(self):
        pass

class MemoryCache(Cache):
    def __init__(self):
        self._data = {}
    
    @property
    def size(self):
        return len(self._data)
    
    def get(self, key):
        return self._data.get(key)
    
    def set(self, key, value):
        self._data[key] = value
    
    def delete(self, key):
        if key in self._data:
            del self._data[key]
    
    def clear(self):
        self._data.clear()

class FileCache(Cache):
    def __init__(self):
        self._data = {}  # Simulated file storage
    
    @property
    def size(self):
        return len(self._data)
    
    def get(self, key):
        return self._data.get(key)
    
    def set(self, key, value):
        self._data[key] = value
        print(f"[FileCache] Written {key} to file")
    
    def delete(self, key):
        if key in self._data:
            del self._data[key]
            print(f"[FileCache] Deleted {key} from file")
    
    def clear(self):
        self._data.clear()
        print("[FileCache] Cleared all files")

# Test
cache = MemoryCache()
cache.set("key1", "value1")
cache.set("key2", "value2")
print(f"Get key1: {cache.get('key1')}")
print(f"Size: {cache.size}")
cache.delete("key1")
print(f"Size after delete: {cache.size}")